# Pandas Cheat Sheet
This notebook is for brief snippets of helpful pandas commands that I've accumulated over time. Because I don't use pandas everyday, I've previously kept all this in an Evernote, but figured a Jupyter notebook would be more appropriate to give examples

## Combining multiple CSVs in a directory into a dataframe
CSVs typically work a bit faster since the read_csv method has the chunksize argument, and read_excel does not. Getting filters in on each chunk before it's concatenated into a combined dataframe makes this a ton more manageable

In [ ]:
import os
import os.path
import pandas as pd
import numpy as np

# if this is a script to be run from a cron job:
# SCRIPT_PATH = os.path.dirname(os.path.abspath(__file__))
# otherwise use '.' because __file__ does not exist in a jupyter notebook

SCRIPT_PATH = '.'
DATA_FOLDER = os.path.join(SCRIPT_PATH, 'data')
CHUNKSIZE = 10000

csvs_in_dir = [os.path.join(DATA_FOLDER, f) for f in os.listdir(DATA_FOLDER) if f.endswith('csv')]

combined_df = pd.DataFrame()

for file in csvs_in_dir:
    for chunk in pd.read_csv(file, chunksize=CHUNKSIZE):
        chunk = chunk.replace(np.nan, 'None')
        # Add any other filters here
        combined_df = pd.concat([combined_df, chunk])

## Converting a column to date/time
For whatever reason, our date/time has never been formatted where pandas could auto-detect a datetime dtype. For the string representation, I typically use https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior

In [ ]:
DTTM_STR_FORMAT = '%m/%d/%Y %H:%M'

combined_df['ACTIVITY_DT_TM'] = pd.to_datetime(combined_df['ACTIVITY_DT_TM'], format=DTTM_STR_FORMAT)
combined_df['SERVICE_DT_TM'] = pd.to_datetime(combined_df['SERVICE_DT_TM'], format=DTTM_STR_FORMAT)

## Replacing column values with regex
See also: https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.replace.html
This is helpful if you need to make sweeping changes or formatting changes to the values of a column

In [ ]:
# Probably not the best way:
pd.options.mode.chained_assignment = None
df['ORDERED_AS_MNEMONIC.1'].replace(regex=True, inplace=True, to_replace=r' \(ONC\)', value=r'')b

## Using the group by function to visualize continuous values

In [ ]:
df.groupby(pd.cut(df3['DIFF'], np.arange( -1830, 200, 20)))['DIFF'].count()
# ['DIFF'] includes what columns to return
# count() is the aggregation function
# pd.cut separates out the continuous values by the range denoted by np.arange()

## Quick aggregation/summarizing columns
Summarizing can be done by creating a ```Series``` or a ```DataFrame```

With a ```Series```, the column in the ```groupby``` method becomes the index, and the column in brackets is subject to the aggregation function

With the ```dataframe```, we can have multiple aggregation functions in the agg method

In [ ]:
# Creating a series
df.groupby('column_name')['column_name'].count()

# Creating a dataframe
df[['column1', 'column2']].groupby('column1').agg(['min', 'max'])

## Sorting values on something that has a MultiIndex
If you have a multi-index, if you do a quick glance at your dataframe, you should see levels in your column names. You can sort things by putting the different level names into a tuple with the sort_values method

In [ ]:
reviews[['variety', 'price']].groupby('variety').agg(['min', 'max']).sort_values([('price', 'min'), ('price', 'max')], ascending=False)

## Grouping and concatenating multiple rows in a columns to a single row
If you have a dozen rows, you can concatenate it to make the final output a little cleaner to end-users. Note this is probably not that helpful for analysts, since they can't do as much with it as a spreadsheet

In [7]:
df = pd.read_csv('sample_settings.csv', header=None, names=['Station', 'Setting'])
df.head()

,Station,Setting
0,Station 1,CriticalCare
1,Station 1,StepDown
2,Station 1,Emergency
3,Station 1,Obstetric
4,Station 1,Behavioral


In [11]:
df.groupby('Station', as_index=False).agg(lambda x: ', '.join(x)).head()

,Station,Setting
0,Station 1,"CriticalCare, StepDown, Emergency, Obstetric, ..."
1,Station 2,"HospitalWide, Pharmacy Only, CriticalCare, Ste..."
2,Station 3,"StepDown, Item Override, HospitalWide, Pharmac..."
3,Station 4,"HospitalWide, Pharmacy Only, Item Override, Ou..."
4,Station 5,"HospitalWide, Pharmacy Only, Item Override, Cr..."


## Exploding out comma-delimited string values into separate rows
This is the opposite of the previous item. This is where you have a comma-delimited string value, that you need to separate into distinct rows

In [29]:
df = pd.read_csv('sample_settings_2.csv')
df.columns = ['ID', 'Setting']
df.head()

,ID,Setting
0,24250002,Pharmacy Only
1,24250003,Pharmacy Only
2,24250005,Pharmacy Only
3,24250006,Pharmacy Only
4,24250007,"Behavioral, Pharmacy Only"


In [30]:
new_df = pd.DataFrame(df['Setting'].str.split(',').tolist(), df['ID']).stack()
new_df.head(7)

ID         
24250002  0     Pharmacy Only
24250003  0     Pharmacy Only
24250005  0     Pharmacy Only
24250006  0     Pharmacy Only
24250007  0        Behavioral
          1     Pharmacy Only
24250008  0     Pharmacy Only
dtype: object

In [31]:
new_df = new_df.reset_index()[['ID', 0]]
new_df.columns = ['ID', 'Setting']
new_df.head(7)

,ID,Setting
0,24250002,Pharmacy Only
1,24250003,Pharmacy Only
2,24250005,Pharmacy Only
3,24250006,Pharmacy Only
4,24250007,Behavioral
5,24250007,Pharmacy Only
6,24250008,Pharmacy Only


## Pivot tables for different time increments and groupings
This assumes that you've already converted a field into a date/time field. Here, we would need to add additional fields to designate the time increment, so that we can run the pivot table off of it

Since we're using lambda functions, we're taking advantage of the attributes and methods of a datetimeindex class. To see the available attributes/methods: https://pandas.pydata.org/pandas-docs/version/0.23.4/generated/pandas.DatetimeIndex.html

**Other fun fact**: For pandas.pivot_table, if you have multiple aggregate functions as a tuple in your aggfunc argument, your resulting pivot table will take the values in your values argument and create a separate column per different aggregate functions

In [4]:
import pandas as pd
refill_df = pd.read_csv('sample_refills.csv')

# Modify the date/time field dtype from object to date/time
DTTM_STR_FORMAT = '%m/%d/%Y %H:%M'
refill_df['TransactionDateTime'] = pd.to_datetime(refill_df['TransactionDateTime'], format=DTTM_STR_FORMAT)

# Create new fields
refill_df['TransactionMonth'] = refill_df['TransactionDateTime'].map(lambda x: x.month)
refill_df['TransactionYear'] = refill_df['TransactionDateTime'].map(lambda x: x.year)
refill_df['TransactionWeek'] = refill_df['TransactionDateTime'].map(lambda x: x.week)
refill_df['TransactionDay'] = refill_df['TransactionDateTime'].map(lambda x: x.date())

pd.pivot_table(refill_df, 
               index=['MedID', 'TransactionYear', 'TransactionWeek'], 
               values=['Quantity'], aggfunc=('sum', 'count')).head()

Quantity    
                                          count sum
MedID  TransactionYear TransactionWeek             
350561 2018            35                     1  10
350918 2018            34                     1   4
                       35                     1   6
350934 2018            34                     2   2
351197 2018            35                     1  20

## Case statements for conditional fields
This is helpful when you have a conditional field but have a ton of conditions. Since python and pandas doesn't have a specific case/switch statement, there is something similar we can create.

By the way, this probably should not be used in this way with numbers, there are better ways to do this. This is just a simple example

In [7]:
import numpy as np

conditions = [
    (refill_df['Quantity'] >= 0) & (refill_df['Quantity'] < 10),
    (refill_df['Quantity'] >= 10) & (refill_df['Quantity'] < 20),
    (refill_df['Quantity'] >= 20) & (refill_df['Quantity'] < 30),
    (refill_df['Quantity'] >= 30) & (refill_df['Quantity'] < 40),
]

choices = [
    'Between 0 and 10',
    'Between 10 and 20',
    'Between 20 and 30',
    'Between 30 and 40',
]

refill_df['new_column'] = np.select(conditions, choices, default='None')

refill_df[['MedID', 'Quantity', 'new_column']].head()

,MedID,Quantity,new_column
0,377309,10,Between 10 and 20
1,472126,10,Between 10 and 20
2,361022,10,Between 10 and 20
3,24250312,20,Between 20 and 30
4,503490,10,Between 10 and 20
